# Image processing demo

See PyMoo:
https://pymoo.org/index.html

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage

In [ ]:
path = "poudre.png"
im = np.asarray(Image.open(path))
plt.figure()
plt.imshow(im)
plt.show()

In [ ]:
imr = im[:300, :, 0]
plt.figure()
plt.imshow(imr, cmap="copper")
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.hist(imr.flatten(), bins=np.arange(256))
plt.grid()
plt.xlabel("Pixel value")
plt.ylabel("Pixel count")
plt.show()

## Thresholding


In [ ]:
thresh = 50.0
imt = (imr > thresh) * 1.0
plt.figure()
plt.imshow(imt, cmap="binary")
plt.colorbar()
plt.show()

In [ ]:
ims = ndimage.gaussian_filter(imr, sigma=2)
ims = (ndimage.sobel(ims, axis=0) ** 2 + ndimage.sobel(ims, axis=1) ** 2) ** 0.5

plt.figure()
plt.imshow(ims, cmap="jet")
plt.colorbar()
plt.show()

In [ ]:
ssize = 31
mid = ssize // 2
structure = np.ones((ssize, ssize), dtype=np.uint8)
for r in range(ssize):
    for c in range(ssize):
        radius = (r - mid) ** 2 + (c - mid) ** 2
        if radius > (ssize / 2) ** 2:
            structure[r, c] = 0

# EROSION
ime = ndimage.binary_erosion(imt, structure=structure)
plt.figure()
plt.imshow(ime, cmap="binary")
plt.colorbar()
plt.show()

In [ ]:
ssize // 2

In [ ]:
imtg = ndimage.gaussian_filter(ims, sigma=5)

plt.figure()
plt.imshow(imtg, cmap="jet")
plt.colorbar()
plt.show()

In [ ]:
import numba

# GENETIC OPTIMIZATION
nr, nc = imt.shape


@numba.jit
def cost(row, col, rad):
    score = 0
    irow = int(row)
    icol = int(col)
    irad = int(rad)
    for r in range(max(irow - irad - 1, 0), min(irow + irad + 1, nr)):
        for c in range(max(icol - irad - 1, 0), min(icol + irad + 1, nc)):
            radius = ((r - row) ** 2 + (c - col) ** 2) ** 0.5
            if radius <= rad:
                pixel = imt[r, c]
                if pixel == 1:
                    score += 1
                elif pixel == 0:
                    score -= 10
    return score

In [ ]:
imf = np.zeros(imt.shape)

rad = 45 / 2
for r in range(nr):
    for c in range(nc):
        imf[r, c] = cost(r, c, rad)

In [ ]:
plt.figure()
plt.imshow(np.where(imf >= 0, imf, np.nan))
plt.colorbar()

In [ ]:
imt.size